In [1]:
with open("test_input.txt", "r") as f:
    test_input = f.read().split("\n")

with open("input.txt", "r") as f:
    real_input = f.read().split("\n")

test_input

['O....#....',
 'O.OO#....#',
 '.....##...',
 'OO.#O....O',
 '.O.....O#.',
 'O.#..O.#.#',
 '..O..#O..O',
 '.......O..',
 '#....###..',
 '#OO..#....']

In [23]:
def get_cube_indices(grid):
    return [(i,j) for i, line in enumerate(grid) for j, c in enumerate(line) if c=="#"]

print(get_cube_indices(test_input))

[(0, 5), (1, 4), (1, 9), (2, 5), (2, 6), (3, 3), (4, 8), (5, 2), (5, 7), (5, 9), (6, 5), (8, 0), (8, 5), (8, 6), (8, 7), (9, 0), (9, 5)]


In [20]:
def calc_load(grid, index):
    load = 0
    current_load = len(grid) - index[0] - 1
    for c in map(lambda s:s[index[1]], grid[index[0]+1:]):
        if c == "#": 
            break
        elif c == "O":
            load += current_load
            current_load -= 1
    
    return load

calc_load(test_input, (-1,2))


10

In [21]:
def calc_total_load(grid):
    square_indices = [(-1, i) for i in range(len(grid[0]))] + get_cube_indices(grid)
    return sum(calc_load(grid, index) for index in square_indices)

calc_total_load(test_input)

136

In [22]:
calc_total_load(real_input)

109654

In [24]:
def split_input(lines):
    return [list(line) for line in lines]

split_input(test_input)

[['O', '.', '.', '.', '.', '#', '.', '.', '.', '.'],
 ['O', '.', 'O', 'O', '#', '.', '.', '.', '.', '#'],
 ['.', '.', '.', '.', '.', '#', '#', '.', '.', '.'],
 ['O', 'O', '.', '#', 'O', '.', '.', '.', '.', 'O'],
 ['.', 'O', '.', '.', '.', '.', '.', 'O', '#', '.'],
 ['O', '.', '#', '.', '.', 'O', '.', '#', '.', '#'],
 ['.', '.', 'O', '.', '.', '#', 'O', '.', '.', 'O'],
 ['.', '.', '.', '.', '.', '.', '.', 'O', '.', '.'],
 ['#', '.', '.', '.', '.', '#', '#', '#', '.', '.'],
 ['#', 'O', 'O', '.', '.', '#', '.', '.', '.', '.']]

In [44]:
def rotate(grid):
    return list(list(line)[::-1] for line in zip(*grid))

rotate(split_input(test_input))

[['#', '#', '.', '.', 'O', '.', 'O', '.', 'O', 'O'],
 ['O', '.', '.', '.', '.', 'O', 'O', '.', '.', '.'],
 ['O', '.', '.', 'O', '#', '.', '.', '.', 'O', '.'],
 ['.', '.', '.', '.', '.', '.', '#', '.', 'O', '.'],
 ['.', '.', '.', '.', '.', '.', 'O', '.', '#', '.'],
 ['#', '#', '.', '#', 'O', '.', '.', '#', '.', '#'],
 ['.', '#', '.', 'O', '.', '.', '.', '#', '.', '.'],
 ['.', '#', 'O', '.', '#', 'O', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '#', '.', '.', '.', '.'],
 ['.', '.', '.', 'O', '#', '.', 'O', '.', '#', '.']]

In [55]:
def stack_stones(grid, index, total):
    for i in range(total):
        grid[index[0]+i+1][index[1]] = "O"
    
def tilt(grid):
    for j in range(len(grid[0])):
        running_count = 0
        for i in range(len(grid)-1, -2, -1):
            c = grid[i][j]
            if c == "#" or i == -1:
                stack_stones(grid, (i,j), running_count)
                running_count = 0
            elif c == "O":
                running_count += 1
                grid[i][j] = "."
    return grid

grid = split_input(test_input)
tilt(grid)

[['O', 'O', 'O', 'O', '.', '#', '.', 'O', '.', '.'],
 ['O', 'O', '.', '.', '#', '.', '.', '.', '.', '#'],
 ['O', 'O', '.', '.', 'O', '#', '#', '.', '.', 'O'],
 ['O', '.', '.', '#', '.', 'O', 'O', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '#', '.'],
 ['.', '.', '#', '.', '.', '.', '.', '#', '.', '#'],
 ['.', '.', 'O', '.', '.', '#', '.', 'O', '.', 'O'],
 ['.', '.', 'O', '.', '.', '.', '.', '.', '.', '.'],
 ['#', '.', '.', '.', '.', '#', '#', '#', '.', '.'],
 ['#', '.', '.', '.', '.', '#', '.', '.', '.', '.']]

In [60]:

def representation(grid):
    # might not be completely unique, but seems unlikely to be an issue
    def block_counts(column):
        rep = []
        count = 0
        for c in column:
            if c == "O":
                count += 1
            elif count != 0:
                rep.append(count)
                count = 0
        if count > 0:
            rep.append(count)
        return rep
    
    return tuple(tuple(block_counts(col)) for col in zip(*grid))


representation(grid)

((4,), (3,), (1, 2), (1,), (1,), (1,), (1,), (1, 1), (), (1, 1))

In [62]:
def perform_cycle(grid):
    new_grid = grid
    for _ in range(4):
        tilt(new_grid)
        new_grid = rotate(new_grid)
    return new_grid

perform_cycle(split_input(test_input))

[['.', '.', '.', '.', '.', '#', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '#', '.', '.', '.', 'O', '#'],
 ['.', '.', '.', 'O', 'O', '#', '#', '.', '.', '.'],
 ['.', 'O', 'O', '#', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', 'O', 'O', 'O', '#', '.'],
 ['.', 'O', '#', '.', '.', '.', 'O', '#', '.', '#'],
 ['.', '.', '.', '.', 'O', '#', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', 'O', 'O', 'O', 'O'],
 ['#', '.', '.', '.', 'O', '#', '#', '#', '.', '.'],
 ['#', '.', '.', 'O', 'O', '#', '.', '.', '.', '.']]

In [96]:
def perform_n_cycles(grid, n):
    layout_history = {representation(grid) : 0}
    new_grid = grid
    for i in range(n):
        if not i%100: print(i)
        new_grid = perform_cycle(new_grid)
        #key = representation(new_grid)
        key = "".join(["".join(row) for row in new_grid])
        if key in layout_history:
            print("loop")
            old_index = layout_history[key]
            loop_size = i - old_index
            print(old_index, i, loop_size)
            remaining_cycles = n - 1 - i
            leftovers = remaining_cycles % loop_size
            print(i + loop_size * (remaining_cycles//loop_size) + leftovers)
            print(i, n, remaining_cycles, leftovers)
            return perform_n_cycles(new_grid, leftovers + 1)
        layout_history[key] = i
    return new_grid

calc_total_load(perform_n_cycles(split_input(test_input), 1000000000))

0
loop
2 9 7
999999999
9 1000000000 999999990 3
0


106

99999990